In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact
from IPython.display import display, HTML
from datetime import datetime

# API Key
api_key = 'f6a79db0fba22353d1625b676200f157'

# Ambil cuaca saat ini
def get_weather(city_name):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {'q': city_name, 'appid': api_key, 'units': 'metric'}
    response = requests.get(url, params=params)
    data = response.json()
    
    if response.status_code == 200:
        icon_code = data['weather'][0]['icon']
        return {
            'Kota': data['name'],
            'Suhu (°C)': data['main']['temp'],
            'Cuaca': data['weather'][0]['description'].capitalize(),
            'Kelembaban (%)': data['main']['humidity'],
            'Angin (m/s)': data['wind']['speed'],
            'Tekanan (hPa)': data['main']['pressure'],
            'Icon': f"http://openweathermap.org/img/wn/{icon_code}@2x.png"
        }
    else:
        return None

# Ambil prediksi 5 hari
def get_forecast(city_name):
    url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {'q': city_name, 'appid': api_key, 'units': 'metric'}
    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200:
        forecast = []
        for item in data['list']:
            forecast.append({
                'Waktu': datetime.fromtimestamp(item['dt']),
                'Suhu (°C)': item['main']['temp'],
                'Kelembaban (%)': item['main']['humidity'],
                'Angin (m/s)': item['wind']['speed']
            })
        return pd.DataFrame(forecast)
    return None

# Tampilkan kartu cuaca cantik
def display_weather_card(data):
    html = f"""
    <div style="background: linear-gradient(135deg, #2980b9, #6dd5fa); 
                padding: 20px; border-radius: 20px; color: white; 
                width: 360px; margin: 15px 0; font-family: 'Arial';">
        <h2>{data['Kota']}</h2>
        <img src="{data['Icon']}" style="float:right; height:60px;">
        <h3>{data['Cuaca']}</h3>
        <p style="font-size:16px; line-height: 1.6;">
            🌡️ Suhu: <b>{data['Suhu (°C)']}°C</b><br>
            💧 Kelembaban: <b>{data['Kelembaban (%)']}%</b><br>
            💨 Angin: <b>{data['Angin (m/s)']} m/s</b><br>
            🔵 Tekanan: <b>{data['Tekanan (hPa)']} hPa</b>
        </p>
    </div>
    """
    display(HTML(html))

# Plot grafik interaktif
def plot_forecast_chart(df, city):
    plt.figure(figsize=(12,6))
    sns.set_theme(style="whitegrid")
    sns.lineplot(x='Waktu', y='Suhu (°C)', data=df, label='Suhu', color='tomato', marker='o')
    sns.lineplot(x='Waktu', y='Kelembaban (%)', data=df, label='Kelembaban', color='royalblue', marker='s')
    sns.lineplot(x='Waktu', y='Angin (m/s)', data=df, label='Angin', color='green', marker='^')
    plt.title(f'Grafik Perkiraan Cuaca 5 Hari di {city}', fontsize=16)
    plt.xlabel('Waktu')
    plt.ylabel('Nilai')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Simpan ke file
def save_to_file(df, filename):
    df.to_csv(f"{filename}.csv", index=False)
    df.to_excel(f"{filename}.xlsx", index=False)
    print(f"📁 Data berhasil disimpan sebagai {filename}.csv dan {filename}.xlsx")

# Dashboard interaktif
def interactive_weather_dashboard():
    city_input = widgets.Text(
        value='Jakarta',
        description='Kota:',
        placeholder='Contoh: Jakarta, Surabaya'
    )
    display(city_input)

    button = widgets.Button(description="Lihat Cuaca 🔍")
    output = widgets.Output()
    display(button, output)

    def on_click(b):
        with output:
            output.clear_output()
            cities = [c.strip() for c in city_input.value.split(',')]
            data_list = []
            for city in cities:
                print(f"🔄 Mengambil data untuk {city}...")
                data = get_weather(city)
                if data:
                    display_weather_card(data)
                    data_list.append(data)
                else:
                    print(f"⚠️ Data tidak ditemukan untuk {city}")

            if data_list:
                df = pd.DataFrame(data_list)
                display(df.style.background_gradient(cmap='coolwarm'))
                save_to_file(df, "cuaca_summary")

                dropdown = widgets.Dropdown(options=cities, description="Detail:")
                display(dropdown)

                def on_dropdown_change(change):
                    if change['type'] == 'change' and change['name'] == 'value':
                        selected = change['new']
                        forecast_df = get_forecast(selected)
                        if forecast_df is not None:
                            plot_forecast_chart(forecast_df, selected)
                        else:
                            print("⚠️ Forecast tidak tersedia.")

                dropdown.observe(on_dropdown_change)

    button.on_click(on_click)

# Jalankan
interactive_weather_dashboard()


Text(value='Jakarta', description='Kota:', placeholder='Contoh: Jakarta, Surabaya')

Button(description='Lihat Cuaca 🔍', style=ButtonStyle())

Output()